<a href="https://colab.research.google.com/github/KaihangZhao/DL/blob/main/CNN_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use pretrained VGG, Alexnet and Resnet models for CIFAR-10 multiclassifications

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
import torchvision

In [ ]:
# training function
def get_CIFAR10(size,batch_size):
    transform=transforms.Compose([transforms.ToTensor(),
                                  transforms.Resize(size)])

    train_ds=datasets.CIFAR10("./data",train=True,download = True,transform = transform)
    train_dl=DataLoader(train_ds,batch_size=batch_size,shuffle = True)
    test_ds=datasets.CIFAR10("./data",train=False,download = True,transform = transform)
    test_dl=DataLoader(test_ds,batch_size=batch_size,shuffle = True)
    return train_dl, test_dl

def train_one_epoch(model, optimizer, train_dl):
    device = "cuda" if torch.cuda.is_available else "cpu"
    train_loss = 0
    for X, y in train_dl:
        model.train()
        X = X.to(device)
        y = y.to(device)
        y_pred = model(X)
        loss = F.cross_entropy(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * X.size(0)
        torch.cuda.empty_cache()
    train_loss = train_loss / len(train_dl.dataset)
    return train_loss


def test(model, test_dl):
    device = "cuda" if torch.cuda.is_available else "cpu"
    test_loss = 0
    accuracy = 0
    for X, y in test_dl:
        X = X.to(device)
        y = y.to(device)
        model.eval()
        y_pred = model(X)
        loss = F.cross_entropy(y_pred, y)

        test_loss += loss.item() * X.size(0)
        accuracy += sum(y_pred.argmax(dim=1) == y)
        torch.cuda.empty_cache()
    # calculate accuracy and loss
    test_loss = test_loss / len(test_dl.dataset)
    accuracy = accuracy / len(test_dl.dataset)
    return test_loss, accuracy.item()


def train_loop(model, optimizer, train_dl, test_dl, epoch):
    for i in range(epoch):
        train_loss = train_one_epoch(model, optimizer, train_dl)
        test_loss, test_acc = test(model, test_dl)
        print(
            f"""train loss:{round(train_loss, 3)}, test loss: {round(test_loss, 3)}, test acc: {round(test_acc, 3)}""")
            

In [ ]:
train_dl,test_dl = get_CIFAR10(size=224,batch_size=50)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
vgg11 = torchvision.models.vgg11(pretrained=True)
linear = nn.Sequential(nn.LeakyReLU(),
                       nn.Linear(1000,10))
model_vgg = nn.Sequential(
    vgg11,
    linear
).cuda()

In [ ]:
# a=0
# for i in model_vgg.parameters():
#   a+=1
#   if a in [23,24]:
#     pass
#   else:
#     i.requires_grad = False

#model=VGG(3,vgg_arch).cuda()
optimizer=torch.optim.AdamW(model_vgg.parameters(),lr=0.00005,weight_decay=5e-4) 
train_loop(model_vgg,optimizer,train_dl,test_dl,4)
optimizer=torch.optim.AdamW(model_vgg.parameters(),lr=0.000001,weight_decay=5e-3) 
train_loop(model_vgg,optimizer,train_dl,test_dl,6)

train loss:0.534, test loss: 0.287, test acc: 0.902
train loss:0.203, test loss: 0.284, test acc: 0.904
train loss:0.095, test loss: 0.278, test acc: 0.916
train loss:0.057, test loss: 0.335, test acc: 0.908
train loss:0.013, test loss: 0.292, test acc: 0.925
train loss:0.006, test loss: 0.297, test acc: 0.927
train loss:0.004, test loss: 0.308, test acc: 0.927
train loss:0.003, test loss: 0.32, test acc: 0.929
train loss:0.002, test loss: 0.327, test acc: 0.929
train loss:0.002, test loss: 0.337, test acc: 0.93


In [ ]:
alex = torchvision.models.alexnet(pretrained=True)
model_alex = nn.Sequential(
    alex,
    linear
).cuda()

In [ ]:
optimizer=torch.optim.AdamW(model_alex.parameters(),lr=0.00005,weight_decay=5e-4) 
train_loop(model_alex,optimizer,train_dl,test_dl,4)
optimizer=torch.optim.AdamW(model_alex.parameters(),lr=0.000001,weight_decay=5e-3) 
train_loop(model_alex,optimizer,train_dl,test_dl,6)

train loss:0.627, test loss: 0.418, test acc: 0.855
train loss:0.364, test loss: 0.363, test acc: 0.876
train loss:0.269, test loss: 0.316, test acc: 0.89
train loss:0.196, test loss: 0.311, test acc: 0.893
train loss:0.103, test loss: 0.267, test acc: 0.913
train loss:0.088, test loss: 0.267, test acc: 0.913
train loss:0.084, test loss: 0.268, test acc: 0.915
train loss:0.079, test loss: 0.267, test acc: 0.916
train loss:0.076, test loss: 0.268, test acc: 0.916
train loss:0.071, test loss: 0.27, test acc: 0.916


In [ ]:
res = torchvision.models.resnet18(pretrained=True)
linear = nn.Sequential(nn.LeakyReLU(),
                       nn.Linear(1000,10))
model_res = nn.Sequential(
    res,
    linear
).cuda()


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
optimizer=torch.optim.AdamW(model_res.parameters(),lr=0.00005,weight_decay=5e-4) 
train_loop(model_res,optimizer,train_dl,test_dl,4)
optimizer=torch.optim.AdamW(model_res.parameters(),lr=0.000001,weight_decay=5e-3) 
train_loop(model_res,optimizer,train_dl,test_dl,6)

train loss:0.372, test loss: 0.199, test acc: 0.931
train loss:0.106, test loss: 0.191, test acc: 0.937
train loss:0.046, test loss: 0.204, test acc: 0.937
train loss:0.033, test loss: 0.225, test acc: 0.936
train loss:0.019, test loss: 0.182, test acc: 0.947
train loss:0.012, test loss: 0.178, test acc: 0.948
train loss:0.008, test loss: 0.174, test acc: 0.948
train loss:0.007, test loss: 0.169, test acc: 0.95
train loss:0.006, test loss: 0.169, test acc: 0.95
train loss:0.005, test loss: 0.17, test acc: 0.951
